In [62]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import json
import geopandas as gpd

In [4]:
tree_df = pd.read_csv('./final_data/trees.csv')
borough_df = pd.read_csv('./final_data/boroughs.csv')
UHF_df = pd.read_csv('./final_data/UHF.csv')

In [148]:

with open('./geo_data/borough.geo.json', 'r') as file:
    boroughs = json.load(file)
with open('./geo_data/UHF34.geo.json', 'r') as file:
    nbds = json.load(file)
borough_df['BoroName'] = borough_df['Borough']
del nbds["features"][0]
nbds
boro_gdf = gpd.GeoDataFrame.from_features(boroughs["features"])
uhf_gdf = gpd.GeoDataFrame.from_features(nbds["features"])
uhf_gdf.set_index('GEOCODE', inplace=True)
uhf_bounds=uhf_gdf.bounds
boro_gdf.set_index('BoroName', inplace=True)
boro_bounds=boro_gdf.bounds


In [154]:
def get_bounds(b_list, x_to_y=0, scale_x=1.1):
    b_box = 0
    if not isinstance(b_list, pd.core.frame.DataFrame):
        b_box = [b_list['minx'], 
            b_list['maxx'],
            b_list['miny'],
            b_list['maxy']]
    else:
        b_box = [min(b_list['minx']), 
            max(b_list['maxx']),
            min(b_list['miny']),
            max(b_list['maxy'])]
    center = [(b_box[1]+b_box[0])/2, (b_box[3]+b_box[2])/2]
    width = b_box[1]-b_box[0]
    height = b_box[3]-b_box[2]
    ratio = width/height
    scale = scale_x
    scale_x = 1
    scale_y = 1
    if x_to_y == 0:
        ()
    elif x_to_y < ratio:
        scale_y = ratio/x_to_y
    elif x_to_y > ratio:
        scale_x = x_to_y/ratio
    new_left = center[0] - width/2 * scale * scale_x
    new_right = center[0] + width/2 * scale * scale_x
    new_top = center[1] + height/2 * scale * scale_y   
    new_bottom = center[1] - height/2 * scale * scale_y
    return {
        "lataxis":{
            "range":[new_bottom,new_top]
        },
        "lonaxis":{
            "range":[new_left,new_right]
        }
    }
    

In [ ]:
# borough num trees map
dataname = 'Number of Trees'
map = go.Figure(data=go.Choropleth(
        geojson=boroughs, # for UHF use `nbds`
        colorscale="Viridis",
        locations=borough_df['BoroName'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.BoroName", #for UHF use "properties.UHF" 
        z=borough_df[dataname] # for UHF use `neighborhood_df[dataname]`
        ))
map.update_layout(
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
        geo=get_bounds(uhf_bounds)
    )


map.show()

<class 'pandas.core.series.Series'>


In [157]:
# borough num trees map
dataname = 'Number of Trees'
map = go.Figure(data=go.Choropleth(
        name='overlay',
        geojson=nbds, # for UHF use `nbds`
        colorscale="Viridis",
        locations=UHF_df['UHF34 Code'], #for UHF use `UHF_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.UHF", #for UHF use "properties.UHF"
        z=UHF_df[dataname], # for UHF use `UHF_df[dataname]`
        zmin=0,
        zmax=max(UHF_df[dataname])
        ))
map.add_trace(
    go.Choropleth(
        name='overlay',
        geojson=nbds, # for UHF use `nbds`
        colorscale="Viridis",
        locations=UHF_df[UHF_df['Borough']=='Bronx']['UHF34 Code'], #for UHF use `UHF_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.UHF", #for UHF use "properties.UHF"
        z=UHF_df[dataname], # for UHF use `UHF_df[dataname]`
        zmin=0,
        zmax=max(UHF_df[dataname]),
        marker=dict(
            line=dict(
                width=2,
                color="aqua"
            )
        )
    )
)
map.update_layout(
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
        geo=get_bounds(boro_bounds.loc['Bronx'], 2)
    )
map.show()

In [160]:
# borough num trees map
dataname = 'Number of Trees'
map = go.Figure(data=go.Choropleth(
        name='overlay',
        geojson=nbds, # for UHF use `nbds`
        colorscale="Viridis",
        locations=UHF_df['UHF34 Code'], #for UHF use `UHF_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.UHF", #for UHF use "properties.UHF"
        z=UHF_df[dataname], # for UHF use `UHF_df[dataname]`
        zmin=0,
        zmax=max(UHF_df[dataname])
        ))
map.add_trace(
    go.Choropleth(
        name='overlay',
        geojson=nbds, # for UHF use `nbds`
        colorscale="Viridis",
        locations=UHF_df[UHF_df['UHF34 Code']==101]['UHF34 Code'], #for UHF use `UHF_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.UHF", #for UHF use "properties.UHF"
        z=UHF_df[dataname], # for UHF use `UHF_df[dataname]`
        zmin=0,
        zmax=max(UHF_df[dataname]),
        marker=dict(
            line=dict(
                width=2,
                color="aqua"
            )
        )
    )
)
map.add_trace(
    go.Scattergeo(
        lon = tree_df[tree_df['UHF34 Code']==101]['Long'],
        lat = tree_df[tree_df['UHF34 Code']==101]['Lat'],
        text = tree_df[tree_df['UHF34 Code']==101]['Status'],
        mode = 'markers',
        marker = dict(
            size = 4,
            opacity = 0.7,
            symbol = 'triangle-up',
            color = tree_df[tree_df['UHF34 Code']==101]['Status'],
            
        )        
        )
)
map.update_layout(
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
        geo=get_bounds(uhf_bounds.loc[101],2)
    )
map.show()

In [45]:
tree_df

,Unnamed: 0,Diameter,Status,Lat,Long,Zip Code,UHF34 Code,UHF34 Neighborhood Name,Borough
0,0,3,2,40.723092,-73.844215,11375,405,Ridgewood/Forest Hills,Queens
1,1,21,2,40.794111,-73.818679,11357,403,Flushing/Clearview,Queens
2,2,3,3,40.717581,-73.936608,11211,201,Greenpoint,Brooklyn
3,3,10,3,40.713537,-73.934456,11211,201,Greenpoint,Brooklyn
4,4,21,3,40.666778,-73.975979,11215,202,Downtown Brooklyn/Heights/Slope,Brooklyn
...,...,...,...,...,...,...,...,...,...
683783,683783,25,3,40.713211,-73.954944,11211,201,Greenpoint,Brooklyn
683784,683784,7,3,40.715194,-73.856650,11375,405,Ridgewood/Forest Hills,Queens
683785,683785,12,3,40.620762,-74.136517,10314,503504,Southern Staten Island,Staten Island
683786,683786,9,3,40.850828,-73.903115,10457,105106107,The South Bronx,Bronx
